
This notebook is an enhanced version to rename the drugs found to more common names. It leverages both the RxNav API (https://mor.nlm.nih.gov/RxNav/) and a drug equivalencies dataset found online (https://data.mendeley.com/datasets/9nmgzttxhm/1).


In [5]:
import pandas as pd
import os
import re
import requests
from tqdm import tqdm

In [70]:

path_drugs = '/Users/jada/Desktop/Project/unbIAsed.Rx/CSVFiles/drugs.csv'

df_drugs = pd.read_csv(path_drugs)
pd.set_option('display.width', 500)
print(df_drugs.head())

   REPORT_ID                      DRUGNAME                  INDICATION_NAME_ENG
0     300198           NOVO VENLAFAXINE XR                           Depression
1     300548               BENADRYL ELIXIR  Product used for unknown indication
2     300488                      ELOXATIN                         Colon cancer
3     301050  TWINJECT 0.3MG AUTO-INJECTOR                Anaphylactic reaction
4     300841               APO-PROPAFENONE  Product used for unknown indication


In [71]:
# get unique drug names in the dataset
unique_drugnames = df_drugs['DRUGNAME'].unique()
print(len(unique_drugnames))

drugname_df = pd.DataFrame(unique_drugnames, columns=['DRUGNAME'])

drugname_df["DRUGNAME"] = drugname_df["DRUGNAME"].fillna('').astype(str)
drugname_df["DRUGNAME"] = drugname_df["DRUGNAME"].str.upper()

# function to clean the drug names removing unncessesary information
def clean_drug_name(drug):
  drug = re.sub(r'\b\d+(\.\d+)?\s*(mg|ml|g|mcg|kg|ug|units|tablets|capsules|pills|iu|tab|unit|u|gm|doses|allergan)\b', '', drug, flags=re.IGNORECASE)
  drug = re.sub(r'\b\d+\b', '', drug)

  drug = drug.replace("(", "").replace(")", "").strip()
  drug = drug.replace("-", " ").replace("&", " ").replace("*", " ").strip()
  drug = drug.replace("[", "").replace("]", "").strip()
  drug = drug.replace("/", " ").replace("\\", " ").strip()
  drug = drug.replace(';', ' ').replace(',', "").strip()
  drug = drug.replace('.', " ").replace(':', " ").strip()
  drug = drug.replace('%', " ").replace('#', " ").replace("+", " ").strip()
  drug = re.sub(r'\bL\s+', '', drug, flags=re.IGNORECASE)
  drug = re.sub(r'\bM\s+', '', drug, flags=re.IGNORECASE)
  drug = re.sub(r'\/.*?\/', '', drug)
  terms_to_remove = [
        'AUTO', 'INJECTOR', 'NOVO', 'APO', 'XR', 'STERILE', 'SOLUTION',
        'INJECTION', 'HCTZ', 'GEN', 'VIAL', 'DEPOT', 'SLOW RELEASE', 'MONTH', 'LIQUID',
        'SPRAY', 'NASAL', 'AEM', 'SYRINGE', 'SYRINGES', 'RBV', 'PRE FILLED', 'DM', 'MENTHOL',
        'COUGH', 'CHEST', 'CONGESTION', 'INHALER', 'PATCH', 'VAGINAL', 'GEL', 'LIQ', 'IV', 'ML', 'XL',
        'TURBUHALER', 'TABLET', 'PLUS', 'CREAM', 'USP', 'TAB', 'TABLETS', 'INJ', 'HCL', 'CD', 'DAY', 'PAD', 'TRD', 'NF',
        'CAPLETS', 'SR', 'PD', 'FOR', 'PLACEBO', 'BP', 'ANTI', 'XA', 'VIALS', 'SUS', 'NAS', 'MD', 'ER', 'CR', 'HCT', 'LAR', 'CAP', 'EC',
        'INJECTABLE', 'SUSPENSION', 'GELS', 'IB', 'EXTRA', 'STRENGTH', 'HFA', 'LIQUI', 'CAPSULE', 'ODT', 'EZ', 'SUS', 'NPH', 'GE', 'SINGLE', 'USE',
        'AUTOINJECTOR', 'CARTRIDGE', 'HC', 'OINTMENT','PREFILLED', 'DISPOSABLE', 'MULTIDOSE', 'COLD', 'AND', 'FLU', 'PRE', 'FILLED', 'PS', 'PRESERVATIVE', 'FREE'
        'INHALATION', 'POWDER', 'ORAL', 'ONLY', 'FILM', 'COATED', 'DS', 'SDF', 'FORMULATION', 'RT', 'PA', 'I V', 'DROPS', 'AEROSOL', 'XC', 'AQUEOUS', 'CONTROLLED',
        'RELEASE', 'WITH', 'ACTUATIONS', 'INFUSION', 'CST', 'MR', 'KIT', 'DILUENT', 'SRT', 'CONTIN', 'CRM', 'CAPSULES', 'SUBL', 'DOSE', 'CAPLET', 'EMERGENCY', 'RINSE', 
        'PREFILLED', 'DELAYED', 'JAMP', 'EYE', 'DROPS', 'TROPICAL', 'MATRIX', 'MAXIMUM', 'SHOT', 'ONCE', 'ARTHRITIS', 'PAIN', 'UNKNOWN', 'BY', 'SUBLINGUAL'
    ]

    # Remove or standardize common terms
  for term in terms_to_remove:
      drug = re.sub(r'\b' + re.escape(term) + r'\b', '', drug, flags=re.IGNORECASE)
  drug = re.sub(r'\s+', ' ', drug)

  words = drug.split()
  seen = set()
  deduped_drug = []
  # remove duplicate words
  for word in words:
        if word.lower() not in seen:
            deduped_drug.append(word)
            seen.add(word.lower())

  return ' '.join(deduped_drug).strip()
  # return drug

drugname_df['DRUGNAME'] = drugname_df['DRUGNAME'].apply(clean_drug_name)
drugname_df = drugname_df.drop_duplicates(subset=['DRUGNAME']) # remove duplicates
print(drugname_df.shape)
pd.set_option('display.width', 500)
print(drugname_df.head(300))
save_directory = '/Users/jada/Desktop/Project/unbIAsed.Rx/CSVFiles'
drugname_df.to_csv(os.path.join(save_directory, 'drugnames.csv'), index=False)

26579
(19470, 1)
                     DRUGNAME
0                 VENLAFAXINE
1             BENADRYL ELIXIR
2                    ELOXATIN
3                    TWINJECT
4                 PROPAFENONE
..                        ...
328                  LOXAPINE
329                  RIVOTRIL
331                    CIALIS
332  NATURAL HRT SWISS HERBAL
333                    CAELYX

[300 rows x 1 columns]


In [72]:
# load equivalencies dataset
drug_equivalencies_path = '/Users/jada/Desktop/Project/unbIAsed.Rx/CSVFiles/drug_equivalencies.csv'
drug_equivalencies = pd.read_csv(drug_equivalencies_path, skiprows=1, encoding="latin1")
drug_equivalencies.drop(columns=["RXCUI", "RXAUI", "SAB", "TTY", "CODE"], inplace =True)
drug_equivalencies["DRUGNAME"] = drug_equivalencies["DRUGNAME"].str.upper()
drug_equivalencies["STR"] = drug_equivalencies["STR"].str.upper()

# Ensure all values in are strings and handle missing values
drug_equivalencies['DRUGNAME'] = drug_equivalencies['DRUGNAME'].fillna('').astype(str)
drug_equivalencies['STR'] = drug_equivalencies['STR'].fillna('').astype(str)

drugname_path = '/Users/jada/Desktop/Project/unbIAsed.Rx/CSVFiles/drugnames.csv'
drugname_df = pd.read_csv(drugname_path)


In [73]:
drug_alternates_cache = {}
# function to get alternates using rxnav api and the equivalencies dataset 
def get_alternate_drug(drug):
    if drug in drug_alternates_cache:
        return drug_alternates_cache[drug]
    url = f'https://rxnav.nlm.nih.gov/REST/approximateTerm.json?term={drug}&option=1'
    response = requests.get(url)
    data = response.json()
    if "approximateGroup" in data and "candidate" in data["approximateGroup"]:
        
        candidates = data["approximateGroup"]["candidate"]
        candidates.sort(key=lambda x: x["rank"])
        for candidate in candidates:
            rxcui = candidate["rxcui"]
            score = float(candidate["score"])
            if score >= 10:
                ingredient_url = f"https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/related.json?tty=IN"
                ingredient_response = requests.get(ingredient_url)
                ingredient_data = ingredient_response.json()
                if "relatedGroup" in ingredient_data and "conceptGroup" in ingredient_data["relatedGroup"] and "conceptProperties" in ingredient_data["relatedGroup"]["conceptGroup"][0]:
                    ingredient = ingredient_data["relatedGroup"]["conceptGroup"][0]['conceptProperties'][0]['name'].upper()
                    drug_alternates_cache[drug] = ingredient
                    return ingredient
            
    # search in the equivalencies dataset in case not found by the api
    for _, row in drug_equivalencies.iterrows():
      if re.search(r'\b%s\b' % drug, row['DRUGNAME']) is not None:
          drug_alternates_cache[drug] = row['STR']
          return row['STR']
      
    drug_alternates_cache[drug] = drug
    return drug


tqdm.pandas()

save_directory = '/Users/jada/Desktop/Project/unbIAsed.Rx/CSVFiles'

# function to apply as chunks
def process_and_save_chunk(df_chunk, chunk_index):
    # Apply the get_alternate_drug function
    df_chunk["Alternate drugname"] = df_chunk["DRUGNAME"].progress_apply(get_alternate_drug)
    # Create the file path for saving the CSV
    csv_file_path = os.path.join(save_directory, f"altdrug_chunk_{chunk_index}.csv")
    # Save the DataFrame chunk to a CSV file
    df_chunk.to_csv(csv_file_path, index=False)
    # Optionally print the chunk
    print(f"Chunk {chunk_index} processed and saved.")
    print(df_chunk.head())  # Print only the first few rows to avoid clutter

# Define the chunk size
chunk_size = 250
starting_chunk_index = 0
# Split the DataFrame into chunks and process each chunk
for chunk_index, start_row in enumerate(range(0, drugname_df.shape[0], chunk_size), start=starting_chunk_index):
    # Get the current chunk
    df_chunk = drugname_df.iloc[start_row:start_row + chunk_size]
    # Process and save the current chunk
    process_and_save_chunk(df_chunk, chunk_index)


100%|██████████| 250/250 [20:42<00:00,  4.97s/it] 
/var/folders/sg/pjszdfvx5rxfvg2gwg3zj2f80000gn/T/ipykernel_62912/267283302.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chunk["Alternate drugname"] = df_chunk["DRUGNAME"].progress_apply(get_alternate_drug)


Chunk 0 processed and saved.
          DRUGNAME Alternate drugname
0      VENLAFAXINE        VENLAFAXINE
1  BENADRYL ELIXIR    DIPHENHYDRAMINE
2         ELOXATIN        OXALIPLATIN
3         TWINJECT        EPINEPHRINE
4      PROPAFENONE        PROPAFENONE


100%|██████████| 250/250 [16:23<00:00,  3.93s/it] 
/var/folders/sg/pjszdfvx5rxfvg2gwg3zj2f80000gn/T/ipykernel_62912/267283302.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chunk["Alternate drugname"] = df_chunk["DRUGNAME"].progress_apply(get_alternate_drug)


Chunk 1 processed and saved.
                              DRUGNAME Alternate drugname
250                      CIPROFLOXACIN      CIPROFLOXACIN
251                             KEPPRA      LEVETIRACETAM
252                           RELISTOR   METHYLNALTREXONE
253                           PREZISTA          DARUNAVIR
254  DIDROCAL ETIDRONATE DISOD CA CARB         ETIDRONATE


100%|██████████| 250/250 [16:03<00:00,  3.85s/it]
/var/folders/sg/pjszdfvx5rxfvg2gwg3zj2f80000gn/T/ipykernel_62912/267283302.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chunk["Alternate drugname"] = df_chunk["DRUGNAME"].progress_apply(get_alternate_drug)


...

Chunk 75 processed and saved.
                                                DRUGNAME         Alternate drugname
18750                                     SLEEP TONIGHT!            DIPHENHYDRAMINE
18751                          VITALUX ARED MULTIVITAMIN  VITALUX ARED MULTIVITAMIN
18752  BORAGO OFFICINALIS DL ALPHA TOCOPHERYL ACETATE...                  VITAMIN E
18753                                             ISOPIT                     ISOPIT
18754                                        COMPOUND T4                COMPOUND T4


100%|██████████| 250/250 [30:43<00:00,  7.38s/it] 
/var/folders/sg/pjszdfvx5rxfvg2gwg3zj2f80000gn/T/ipykernel_62912/267283302.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chunk["Alternate drugname"] = df_chunk["DRUGNAME"].progress_apply(get_alternate_drug)


Chunk 76 processed and saved.
                                                DRUGNAME                Alternate drugname
19000                            NATURAL HEALTH CELLFOOD            PEPPERMINT PREPARATION
19001                                TEST MEDIUM PENFILL        MEDIUM CHAIN TRIGLYCERIDES
19002                                     RANTRAMDOLACEP                    RANTRAMDOLACEP
19003  BABY TEAR FREE SANS LARMES SUNSCREEN LOTION ÉC...                      HYPROMELLOSE
19004                   OMEGA FORT EPA DHA ADRIEN GAGNON  OMEGA FORT EPA DHA ADRIEN GAGNON


100%|██████████| 220/220 [27:32<00:00,  7.51s/it] 
/var/folders/sg/pjszdfvx5rxfvg2gwg3zj2f80000gn/T/ipykernel_62912/267283302.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_chunk["Alternate drugname"] = df_chunk["DRUGNAME"].progress_apply(get_alternate_drug)


Chunk 77 processed and saved.
                  DRUGNAME  Alternate drugname
19250            PRISMOCAL           PRISMOCAL
19251            PRILOOSEC           PRILOOSEC
19252        REVISION CORE       REVISION CORE
19253  FLOMAX MORNIFLUMATE        MORNIFLUMATE
19254   MONOTHERAPY EPIVAL  MONOTHERAPY EPIVAL


In [65]:
# drug_alternates_cache = {}
# testing the function for individual drugs
def get_alternate_drug(drug):
    if drug in drug_alternates_cache:
        print('hello')
        return drug_alternates_cache[drug]
    url = f'https://rxnav.nlm.nih.gov/REST/approximateTerm.json?term={drug}&option=1'
    response = requests.get(url)
    data = response.json()
    print(data)
    print(data)
    if "approximateGroup" in data and "candidate" in data["approximateGroup"]:
        
        candidates = data["approximateGroup"]["candidate"]
        print(candidates)
        candidates.sort(key=lambda x: x["rank"])
        for candidate in candidates:
            rxcui = candidate["rxcui"]
            print(rxcui)
            ingredient_url = f"https://rxnav.nlm.nih.gov/REST/rxcui/{rxcui}/related.json?tty=IN"
            ingredient_response = requests.get(ingredient_url)
            ingredient_data = ingredient_response.json()
            print(ingredient_data)
            if "relatedGroup" in ingredient_data and "conceptGroup" in ingredient_data["relatedGroup"] and "conceptProperties" in ingredient_data["relatedGroup"]["conceptGroup"][0]:
                print("ya")
                ingredient = ingredient_data["relatedGroup"]["conceptGroup"][0]['conceptProperties'][0]['name'].upper()
                drug_alternates_cache[drug] = ingredient
                return ingredient
    drug_alternates_cache[drug] = drug
    return drug

alt_drug = get_alternate_drug("TWINJECT")
print(alt_drug)

In [74]:
# concatenate all chunks together into a single csv file
get_directory = '/Users/jada/Desktop/Project/unbIAsed.Rx/CSVFiles'
save_directory = '/Users/jada/Desktop/Project/unbIAsed.Rx/CSVFiles'
output_file = os.path.join(save_directory, 'AlternateDrugs.csv')

# List to store DataFrames
df_list = []

# Iterate over all files in the save directory
for filename in sorted(os.listdir(get_directory)):
    if filename.startswith('altdrug_chunk_') and filename.endswith('.csv'):
        file_path = os.path.join(get_directory, filename)
        # Read the CSV file into a DataFrame
        df_chunk = pd.read_csv(file_path)
        # Append the DataFrame to the list
        df_list.append(df_chunk)
        print(f"Processed {filename}")

# Concatenate all DataFrames in the list
combined_df = pd.concat(df_list, ignore_index=True)

# Save the combined DataFrame to a single CSV file
combined_df.to_csv(output_file, index=False)

Processed altdrug_chunk_0.csv
Processed altdrug_chunk_1.csv
Processed altdrug_chunk_10.csv
Processed altdrug_chunk_11.csv
Processed altdrug_chunk_12.csv
Processed altdrug_chunk_13.csv
Processed altdrug_chunk_14.csv
Processed altdrug_chunk_15.csv
Processed altdrug_chunk_16.csv
Processed altdrug_chunk_17.csv
Processed altdrug_chunk_18.csv
Processed altdrug_chunk_19.csv
Processed altdrug_chunk_2.csv
Processed altdrug_chunk_20.csv
Processed altdrug_chunk_21.csv
Processed altdrug_chunk_22.csv
Processed altdrug_chunk_23.csv
Processed altdrug_chunk_24.csv
Processed altdrug_chunk_25.csv
Processed altdrug_chunk_26.csv
Processed altdrug_chunk_27.csv
Processed altdrug_chunk_28.csv
Processed altdrug_chunk_29.csv
Processed altdrug_chunk_3.csv
Processed altdrug_chunk_30.csv
Processed altdrug_chunk_31.csv
Processed altdrug_chunk_32.csv
Processed altdrug_chunk_33.csv
Processed altdrug_chunk_34.csv
Processed altdrug_chunk_35.csv
Processed altdrug_chunk_36.csv
Processed altdrug_chunk_37.csv
Processed al

In [75]:
# Make a dictionary with the name of the drug and its alternate name

drug_alternates_df_path = '/Users/jada/Desktop/Project/unbIAsed.Rx/CSVFiles/AlternateDrugs.csv'
drug_alternates_df = pd.read_csv(drug_alternates_df_path)

drug_alternates_dict = dict(zip(drug_alternates_df['DRUGNAME'], drug_alternates_df['Alternate drugname']))
print(drug_alternates_dict)


{'VENLAFAXINE': 'VENLAFAXINE', 'BENADRYL ELIXIR': 'DIPHENHYDRAMINE', 'ELOXATIN': 'OXALIPLATIN', 'TWINJECT': 'EPINEPHRINE', 'PROPAFENONE': 'PROPAFENONE', 'EXELON': 'RIVASTIGMINE', 'MORPHINE SULFATE': 'MORPHINE', 'HUMIRA': 'ADALIMUMAB', 'FORTEO': 'TERIPARATIDE', 'LAMICTAL': 'LAMOTRIGINE', 'INFUFER': 'IRON', 'ADDERALL': 'DEXTROAMPHETAMINE', 'CHAMPIX': 'VARENICLINE', 'LYRICA': 'PREGABALIN', 'TAXOTERE': 'DOCETAXEL', 'FENTANYL': 'FENTANYL', 'CESIUM CHLORIDE': 'CESIUM CHLORIDE', 'PEGASYS': 'PEGINTERFERON ALFA-2A', 'LITHIUM': 'LITHIUM', 'TRAZODONE': 'TRAZODONE', 'PARACETAMOL': 'ACETAMINOPHEN', 'RATIO FENTANYL': 'FENTANYL', 'CYCLOBENZAPRINE': 'CYCLOBENZAPRINE', 'REMICADE': 'INFLIXIMAB', 'CLOZAPINE': 'CLOZAPINE', 'EPREX': 'EPOETIN ALFA', 'FERRLECIT': 'SODIUM FERRIC GLUCONATE COMPLEX', 'HEPARIN SODIUM': 'HEPARIN', 'FOSAMAX': 'ALENDRONATE', 'REBIF': 'INTERFERON BETA-1A', 'COUMADIN': 'WARFARIN', 'TIMOLOL MALEATE OPHTHALMIC': 'TIMOLOL', 'ISOVUE': 'IOPAMIDOL', 'NOLVADEX': 'TAMOXIFEN', 'PROSCAR': 'FIN

In [78]:
drugs_df = pd.read_csv('/Users/jada/Desktop/Project/drugs.csv')
drugs_df["DRUGNAME"] = drugs_df["DRUGNAME"].fillna('').astype(str)
drugs_df["DRUGNAME"] = drugs_df["DRUGNAME"].str.upper()

tqdm.pandas()
# clean the drug names in original df to map 
drugs_df['DRUGNAME'] = drugs_df['DRUGNAME'].progress_apply(clean_drug_name)


  0%|          | 1/2988926 [00:00<18:07:09, 45.82it/s]


TypeError: 'dict' object is not callable

In [79]:
# define a function to map the alternate drug names
def map_alternate_drug(drug):
    return drug_alternates_dict.get(drug, drug)

tqdm.pandas()

drugs_df['ALTERNATE DRUG'] = drugs_df['DRUGNAME'].progress_map(map_alternate_drug)

100%|██████████| 2988926/2988926 [00:05<00:00, 577311.43it/s]


In [80]:
print(drugs_df.head())

   REPORT_ID         DRUGNAME                  INDICATION_NAME_ENG   ALTERNATE DRUG
0     300198      VENLAFAXINE                           Depression      VENLAFAXINE
1     300548  BENADRYL ELIXIR  Product used for unknown indication  DIPHENHYDRAMINE
2     300488         ELOXATIN                         Colon cancer      OXALIPLATIN
3     301050         TWINJECT                Anaphylactic reaction      EPINEPHRINE
4     300841      PROPAFENONE  Product used for unknown indication      PROPAFENONE


In [81]:
save_directory = '/Users/jada/Desktop/Project/unbIAsed.Rx/CSVFiles/'
drugs_df.to_csv(os.path.join(save_directory, 'drugs_alternates.csv'), index=False)